In [2]:
!pip install numpy

In [3]:
!pip install pandas

In [4]:
!pip install scikit-learn

In [5]:
!pip install opencv-python

In [6]:
!pip install tensorflow

In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [8]:
def load_images_from_folder(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)
    return images, labels

In [9]:
# Load train dataset
train_cats_path = "train/cats"
train_dogs_path = "train/dogs"
train_images, train_labels = [], []

cats_images, cats_labels = load_images_from_folder(train_cats_path, 0)
dogs_images, dogs_labels = load_images_from_folder(train_dogs_path, 1)

train_images.extend(cats_images)
train_images.extend(dogs_images)
train_labels.extend(cats_labels)
train_labels.extend(dogs_labels)

# Convert to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [10]:
# Load test dataset similarly
test_cats_path = "test/cats"
test_dogs_path = "test/dogs"
test_images, test_labels = [], []

cats_images, cats_labels = load_images_from_folder(test_cats_path, 0)
dogs_images, dogs_labels = load_images_from_folder(test_dogs_path, 1)

test_images.extend(cats_images)
test_images.extend(dogs_images)
test_labels.extend(cats_labels)
test_labels.extend(dogs_labels)

# Convert to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [12]:
# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

In [13]:
# Create a model to output the feature maps
model = Model(inputs=base_model.input, outputs=base_model.output)

In [14]:
# Extract features for train and test datasets
train_features = model.predict(train_images)
test_features = model.predict(test_images)

625/625 ━━━━━━━━━━━━━━━━━━━━ 518s 827ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 132s 844ms/step


In [15]:
# Flatten the features to be used in SVM
train_features = train_features.reshape(train_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [1]:
svm_model = SVC(kernel='linear')
svm_model.fit(train_features, train_labels)

NameError: name 'SVC' is not defined

In [ ]:
# Predict on the test set
test_predictions = svm_model.predict(test_features)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(test_labels, test_predictions)
report = classification_report(test_labels, test_predictions)

In [ ]:
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

In [ ]:
def preprocess_image(image_path, img_size=(64, 64)):
    # Load image
    img = cv2.imread("test3 photo.jpg")
    if img is not None:
        # Resize image
        img = cv2.resize(img, (256,256))
        # Normalize image
        img = img / 255.0
        # Expand dimensions to match the input shape of the model
        img = np.expand_dims(img, axis=0)
    return img

def extract_features(model, img):
    # Extract features using the pre-trained VGG16 model
    features = model.predict(img)
    # Flatten the features to be used in SVM
    features = features.reshape(features.shape[0], -1)
    return features

def predict_image(image_path, feature_extractor_model, svm_model):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Extract features
    features = extract_features(feature_extractor_model, img)
    # Predict using the trained SVM model
    prediction = svm_model.predict(features)
    return prediction[0]

# Paths to the new images to be predicted
new_image_path = "test3 photo.jpg"

# Predict the class of the new image
prediction = predict_image(new_image_path, model, svm_model)

# Output the result
if prediction == 0:
    print("The image is a Cat.")
else:
    print("The image is a Dog.")

plt.imshow(new_image_path)
